# 本番

In [1]:
import os
import sys
import json
import time
from datetime import datetime
import subprocess
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import DataLoader, random_split
from torchvision import datasets, transforms
from torchvision.models import resnet18
from torchvision.transforms import v2
from timm.scheduler import CosineLRScheduler
import onnxruntime
from onnxruntime.quantization import (
    quantize_dynamic, quantize_static, QuantType, QuantFormat, CalibrationDataReader
)
from typing import Union, Dict


c:\Users\thyt\Pytorch\pytorchenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 特定のパスをシステムパスに追加

path = "C:/Users/thyt/confidential_files/Postgresql"
sys.path.append(f'{path}')

with open(f'{path}/config.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

# 設定ファイルからPATHを読み込む
RAW_MODELS_PATH = config['RAW_MODELS_PATH']
PREPROCESSED_MODELS_PATH = config['PREPROCESSED_MODELS_PATH']
QUANTIZED_MODELS_PATH = config['QUANTIZED_MODELS_PATH']

with open(f'{path}/config.json', 'r', encoding='utf-8') as file:
    config = json.load(file)

# 設定ファイルからPATHを読み込む
RAW_MODELS_PATH = config['RAW_MODELS_PATH']
PREPROCESSED_MODELS_PATH = config['PREPROCESSED_MODELS_PATH']
QUANTIZED_MODELS_PATH = config['QUANTIZED_MODELS_PATH']

In [2]:
class CFG:
    commit_hash="AAAAAA"
    
    # 実行日時
    now = datetime.now()
    formatted_date = now.strftime("%Y%m%d_%H%M%S")


    num_workers=2
    n_epochs = 10  # エポック数の設定
    n_epochs = 1  # エポック数の設定
    
    # デバイスの設定
    device = "cuda" if torch.cuda.is_available() else "cpu"
    providers = ["CUDAExecutionProvider"]

path = "C:/Users/thyt/Learning/Learning_py/Repositories/ML_PipeLine_quantization_models/configs"
sys.path.append(f'{path}')

# configモジュールからseed_everything関数をインポート
from config import seed_everything

# 乱数生成器のシードを設定
seed_everything()


ModuleNotFoundError: No module named 'config'

In [4]:
# データの水増しの設定
# データの水増し（データ拡張）のための設定
transform = transforms.Compose([
    transforms.ToTensor(),  # 画像をPyTorchテンソルに変換
    transforms.RandomResizedCrop(32, scale=(0.9, 1.0), antialias=False),  # 画像をランダムにリサイズし、32x32ピクセルにクロップ
    transforms.RandomRotation(degrees=(-15, 15)),  # 画像をランダムに-15度から15度の間で回転
    transforms.RandomHorizontalFlip(),  # 画像をランダムに水平方向に反転
    transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),  # 画像を正規化（平均0.5、標準偏差0.5で各チャンネルを正規化）
])

# CutMixの設定：画像の一部を別の画像で置き換える手法
cutmix = v2.CutMix(num_classes=10)
# MixUpの設定：2つの画像を重ね合わせて混合する手法
mixup = v2.MixUp(num_classes=10, alpha=0.2)
# CutMixとMixUpのうちどちらかをランダムに選択する
cutmix_or_mixup = v2.RandomChoice([cutmix, mixup])

# データのダウンロードとデータの読み込みとデータセットの作成
dataset = datasets.CIFAR10(root="./data", train=True, download=True, transform=transform)

# 8:2 に分割し、それぞれ訓練用と検証用とする
train_dataset, val_dataset = random_split(dataset, [0.8 , 0.2], generator=torch.Generator().manual_seed(42))

Files already downloaded and verified


# データローダーの作成

In [5]:
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=CFG.num_workers)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=CFG.num_workers)



# モデルの作成と損失関数の定義
model = resnet18(num_classes=10).to(CFG.device)
criterion = nn.CrossEntropyLoss(label_smoothing=0.1)

# 最適化手法の定義と学習率スケジューラーの設定
optimizer = torch.optim.Adam(model.parameters())
scheduler = CosineLRScheduler(optimizer, t_initial=CFG.n_epochs, lr_min=1e-5, warmup_t=5, warmup_lr_init=1e-6, warmup_prefix=True)

Cosine annealing scheduler will have no effect on the learning rate since t_initial = t_mul = eta_mul = 1.


# モデルの学習と検証


In [6]:
for epoch in range(CFG.n_epochs):
    # 学習
    model.train()
    scheduler.step(epoch)  # 学習率更新
    for data, targets in train_loader:
        data, targets = data.to(CFG.device), targets.to(CFG.device)
        # CutUp or MixUp の適用
        data, targets = cutmix_or_mixup(data, targets)  # cutmix_or_mixup 関数は実装してください
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, targets)
        loss.backward()
        optimizer.step()

    # 検証
    model.eval()
    correct = 0
    total_loss = 0
    with torch.no_grad():
        for data, targets in val_loader:
            data, targets = data.to(CFG.device), targets.to(CFG.device)
            output = model(data)
            total_loss += criterion(output, targets)
            pred = output.argmax(axis=1)
            correct += (pred == targets).sum().item()
    # 検証結果の表示
    print(f"Epoch 数:{epoch + 1}, Accuracy (val): {correct / len(val_dataset)}, Loss: {total_loss / len(val_loader)}")


Epoch 数:1, Accuracy (val): 0.1196, Loss: 2.409409761428833


In [7]:
# print(data.shape)
# > torch.Size([16, 3, 32, 32])

## 3.2 量子化の準備

In [8]:
# まずはPyTorch形式からONNX形式に変換
x = torch.randn(1, 3, 32, 32, requires_grad=True)

# PyTorchモデルをONNX形式にエクスポート
torch.onnx.export(
    model.cpu().eval(),  # モデルを評価モードに設定,  # PyTorchモデル
    x,  # モデルへの入力
    # "model12.onnx ", # ONNX 形式に変換されたモデルの保存先ファイルパス
    f"{RAW_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model.onnx", # ONNX 形式に変換されたモデルの保存先ファイルパス
    export_params=True,  # モデルの保存時にパラメータを含めるかどうか
    opset_version=12,  # 演算子のバージョン
    do_constant_folding=True,  # 定数入力を含む演算の一部を事前に計算された定数ノードに変換するかどうか
    input_names=["input"],  # モデルの入力に対する任意の名前
    output_names=["output"],  # モデルの出力に対する任意の名前
    dynamic_axes={"input": {0: "batch_size"},
                   "output": {0: "batch_size"}}  # 入力と出力の可変軸
)

# 次はモデルの前処理（モデルの最適化）を行います。
input_path = f"{RAW_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model.onnx"
output_path = f"{PREPROCESSED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer.onnx"

!python -m onnxruntime.quantization.preprocess --input {input_path} --output {output_path}

## 3.3 量子化の実行
- 動的量子化と静的量子化（QDQ とQOperator）を行う

### 動的量子化

In [9]:
# 次はモデルの前処理（モデルの最適化）を行います。
input_path = f"{PREPROCESSED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer.onnx"
output_path = f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_dq_qop_wa_u8u8.onnx"

# wa_u8u8 は重みがu 8 、アクティベーションがu8 を表している
quantize_dynamic(input_path,  # 前処理後のONNXモデルのパス
        output_path,  # 量子化後のONNXモデルの保存先パス
        weight_type=QuantType.QUInt8  # 重みの量子化方式を指定
        )

### QDQ での静的量子化

In [10]:
# キャリブレーション用データリーダーのクラス定義
class ImgDataReader(CalibrationDataReader):
    def __init__(self, imgs: np.ndarray) -> None:
        self.img_dicts = iter([{"input": img[None]} for img in imgs])

    def get_next(self) -> Union[Dict[str, np.ndarray], None]:
        # get_next 関数は{"input": ndarray 形式} で返す、全て返し終わったらNone
        return next(self.img_dicts, None)

# ラベルごとに画像を仕分ける
label_imgs_dict = dict(zip(range(10), [[] for i in range(10)]))

for img, target in val_dataset:
    label_imgs_dict[target].append(img.numpy())

# 各ラベルを30 ずつとするデータを作成
calibration_data = []

for i in range(10):
    calibration_data.extend(label_imgs_dict[i][:30])

calibration_data = np.asarray(calibration_data)  # (300 , 3, 32, 32)

In [11]:
calibration_data.shape

(300, 3, 32, 32)

In [12]:
data_reader = ImgDataReader(calibration_data)
quant_format = QuantFormat.QDQ

input_path = f"{PREPROCESSED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer.onnx"
output_path = f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_sq_qdq_wa_s8s8.onnx"

# s8 は符号付き8bit 整数、u8 はデータ型が符号なし8bit 整数を表す
quantize_static(
    input_path,  # 入力のONNXモデルのパス
    output_path,  # 量子化後のONNXモデルの保存先パス
    data_reader,
    quant_format=quant_format,
    activation_type=QuantType.QUInt8
)

### QOperatorにおける静的量子化

In [13]:
data_reader = ImgDataReader(calibration_data)
quant_format = QuantFormat.QOperator


input_path = f"{PREPROCESSED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer.onnx"
output_path = f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_sq_qop_wa_s8u8.onnx"

# s8 は符号付き8bit 整数、u8 はデータ型が符号なし8bit 整数を表す
quantize_static(
    input_path,  # 入力のONNXモデルのパス
    output_path,  # 量子化後のONNXモデルの保存先パス
    data_reader,
    quant_format=quant_format,
    activation_type=QuantType.QUInt8
)

## 3.4 ファイルサイズの計算


In [14]:
# 量子化前後のファイルサイズをMB 単位で確認

# 量子化前の前処理済みモデルのファイルサイズ
print(os.path.getsize(f"{PREPROCESSED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer.onnx") / (1000**2), "MB")
# 動的量子化済みモデルのファイルサイズ
print(os.path.getsize(f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_dq_qop_wa_u8u8.onnx") / (1000**2), "MB")
# 静的量子化（QDQ）済みモデルのファイルサイズ
print(os.path.getsize(f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_sq_qdq_wa_s8s8.onnx") / (1000**2), "MB")
# 静的量子化（QOperator）済みモデルのファイルサイズ
print(os.path.getsize(f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_sq_qop_wa_s8u8.onnx") / (1000**2), "MB")


44.720849 MB
11.234565 MB
11.236189 MB
11.217224 MB


## 3.5 推論の実行

In [15]:
"""input_path には前処理済みモデル、動的量子化済みモデル、静的量子化済みモデル（QOperator）
ファイルパスをそれぞれ与えて以下の推論を実行します。
推論はGoogleのColaboratoryのCPU環境（2.20GHzのXeonで拡張命令セット
としてAVX2 を持つ）にて実行しました。"""

models_path = [
    f"{PREPROCESSED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer.onnx",
    f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_dq_qop_wa_u8u8.onnx",  # 動的量子化済みモデルのファイルサイズ
    f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_sq_qdq_wa_s8s8.onnx", # 静的量子化（QDQ）済みモデルのファイルサイズ
    f"{QUANTIZED_MODELS_PATH}/{CFG.commit_hash}_{CFG.formatted_date}_model_infer_sq_qop_wa_s8u8.onnx", # 静的量子化（QOperator）済みモデルのファイルサイズ
      ]

# 空のDataFrameを作成
columns = ['モデルの条件','ファイルサイズ/MB', '正解率/%', '推論時間/s']
df = pd.DataFrame(columns=columns)

for input_path in models_path:

    session_fp32 = onnxruntime.InferenceSession(input_path, providers=CFG.providers)

    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

    test_dataset = datasets.CIFAR10(root="./data", train=False, download=True, transform=transform)
    test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=CFG.num_workers)

    # 推論と時間計測
    correct = 0
    start_time = time.perf_counter()

    for data, targets in test_loader:
        data, targets = data.numpy(), targets.numpy()
        output = session_fp32.run([], {"input": data})[0]  # 推論の実行
        pred = output.argmax(axis=1)
        correct += (pred == targets).sum()
    
    end_time = time.perf_counter()
    
    new_record ={
        "モデルの条件":input_path,
        "ファイルサイズ/MB":(str(os.path.getsize(f"{input_path}") / (1000**2))+ " MB"),
        "正解率/%":(correct / len(test_dataset)),
        "推論時間/s":(end_time - start_time)
        }
    df = pd.concat([df, pd.DataFrame([new_record])],axis=0)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## 3.6 結果

In [16]:
df

,モデルの条件,ファイルサイズ/MB,正解率/%,推論時間/s
0,C:/Users/thyt/Learning/Learning_py/Repositorie...,44.720849 MB,0.1285,7.978916
0,C:/Users/thyt/Learning/Learning_py/Repositorie...,11.234565 MB,0.1274,25.075862
0,C:/Users/thyt/Learning/Learning_py/Repositorie...,11.236189 MB,0.1275,7.917494
0,C:/Users/thyt/Learning/Learning_py/Repositorie...,11.217224 MB,0.1276,6.656830


1. ファイルサイズの比較：
   - 最初のモデル（`model_infer.onnx`）は約44.72MBで、他のモデルに比べて大きいです。
   - 残りの3つのモデルは約11.23MBで、かなりサイズが小さくなっています。

2. 正解率の比較：
   - すべてのモデルの正解率は似ており、約0.128（12.8%）です。
   - 最高の正解率は`model_infer_dq_qop_wa_u8u8.onnx`の0.1297（12.97%）で、最低は`model_infer_sq_qdq_wa_s8s8.onnx`の0.1275（12.75%）です。
   - これらの差は非常に小さいため、モデル間の正解率に大きな差はないと言えます。

3. 推論時間の比較：
   - 最初のモデル（`model_infer.onnx`）と`model_infer_sq_qdq_wa_s8s8.onnx`の推論時間はほぼ同じで、約8秒です。
   - `model_infer_sq_qop_wa_s8u8.onnx`は最も推論時間が短く、約7.12秒です。
   - `model_infer_dq_qop_wa_u8u8.onnx`は、推論に約24.41秒とかなり長い時間がかかります。

- 総合的な考察：
   - ファイルサイズを大幅に削減しても、正解率には大きな影響が見られません。
   - ただし、`model_infer_dq_qop_wa_u8u8.onnx`は他のモデルに比べて推論時間が大幅に長くなっています。これは、サイズ削減のための最適化が推論速度に影響を与えている可能性があります。
   - `model_infer_sq_qop_wa_s8u8.onnx`は、ファイルサイズを大幅に削減しつつ、推論時間を短縮しているため、最適なバランスを提供している可能性があります。

In [17]:
import winsound

# ビープ音を再生する
winsound.Beep(1000, 500)  # 周波数1000Hzで500ミリ秒のビープ音

# WAVファイルを再生する
winsound.PlaySound("sound.wav", winsound.SND_FILENAME)
